Parcourt train/P*/T*/.
Calcule le volume de chaque lésion avec SimpleITK.
(Optionnel) Compare avec un fichier CSV de référence venant du dataset.
Sauvegarde tout dans un CSV pour vérifier les différences.

In [ ]:
import os
import csv
import SimpleITK as sitk

# Dossier racine contenant tous les patients
BASE_DIR = "../../27919209/MSLesSegDataset/train"  # adapte si besoin

# (Optionnel) CSV de référence fourni par le dataset,
# contenant par exemple: patient,session,lesion_id,volume_ml
REFERENCE_CSV = None  # ex: "../../27919209/MSLesSegDataset/lesion_volumes_reference.csv"

# Chargement éventuel des volumes de référence dans un dictionnaire
ref_volumes = {}  # clé: (patient, session, lesion_id) -> volume_ml_ref

if REFERENCE_CSV is not None:
    with open(REFERENCE_CSV, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            key = (row["patient"], row["session"], row["lesion_id"])
            ref_volumes[key] = float(row["volume_ml"])

# Fichier de sortie
output_csv = "lesion_volumes_sitk_vs_ref.csv"

with open(output_csv, mode="w", newline="") as f_out:
    writer = csv.writer(f_out)
    writer.writerow([
        "patient",
        "session",
        "lesion_id",
        "voxel_count",
        "volume_mm3",
        "volume_ml",
        "volume_ml_ref",        # NaN si pas de référence
        "volume_diff_ml"        # SITK - ref
    ])

    # Boucle sur les patients
    for patient in sorted(os.listdir(BASE_DIR)):
        patient_dir = os.path.join(BASE_DIR, patient)
        if not os.path.isdir(patient_dir):
            continue

        # Boucle sur les sessions (T1, T2, T3, etc.)
        for session in sorted(os.listdir(patient_dir)):
            session_dir = os.path.join(patient_dir, session)
            if not os.path.isdir(session_dir):
                continue

            prefix = f"{patient}_{session}"
            mask_path = os.path.join(session_dir, f"{prefix}_MASK.nii.gz")

            if not os.path.exists(mask_path):
                print(f"[WARN] Pas de masque pour {patient} / {session}, on saute.")
                continue

            print(f"Traitement volumes: {patient} / {session} ...")

            # Lecture du masque
            mask_img = sitk.ReadImage(mask_path)

            # Masque binaire : lésion = voxels > 0
            lesion_bin = mask_img > 0

            # Composantes connexes 3D : chaque lésion reçoit un label distinct (1,2,3,...)
            lesion_cc = sitk.ConnectedComponent(lesion_bin)

            # Stats par label (pour obtenir le nombre de voxels)
            stats = sitk.LabelStatisticsImageFilter()
            stats.Execute(lesion_cc, lesion_cc)  # image d'intensité = image de labels

            # Volume d'un voxel (en mm^3)
            sx, sy, sz = mask_img.GetSpacing()
            voxel_vol_mm3 = sx * sy * sz

            for lesion_id in stats.GetLabels():
                if lesion_id == 0:
                    continue  # 0 = fond

                voxel_count = stats.GetCount(lesion_id)
                volume_mm3 = voxel_vol_mm3 * voxel_count
                volume_ml = volume_mm3 / 1000.0

                key = (patient, session, str(int(lesion_id)))

                if key in ref_volumes:
                    volume_ml_ref = ref_volumes[key]
                    volume_diff_ml = volume_ml - volume_ml_ref
                else:
                    volume_ml_ref = float("nan")
                    volume_diff_ml = float("nan")

                writer.writerow([
                    patient,
                    session,
                    int(lesion_id),
                    int(voxel_count),
                    float(volume_mm3),
                    float(volume_ml),
                    volume_ml_ref,
                    volume_diff_ml
                ])

print(f"Terminé. Résultats sauvegardés dans {output_csv}")
